# Heart Disease Prediction Model - Exploratory Analysis #
## Author: Madison Little ##
## Date: Sep. 4, 2024 ##

This initial report explores the data's quality, completeness, and distribution.

In [320]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json

In [321]:
# The mappings file contains expanded versions of the 
# categorical values found in the data.  This helps the 
# charts to be more readable.
with open('data/categorical_mappings.json', 'r') as f:
    mappings = json.load(f)

The following two defined functions are used throughout the report to create similar charts for each of the categorical values.
I defined them in this notebook so I would not have to read the categorical_mapping.json file each time the function is called or pass the json data as a parameter to the function.  This way, the functions can access the mapping as a global variable.

In [322]:
def categorical_ct(df:pd.DataFrame, field:str):
    '''
    Chart the frequency of each category within a categorical field.

    Input: dataframe, field name, chart title, and axis titles
    Output: bar chart
    '''
    data = df[field].replace(mappings[field]).value_counts().to_dict()
    fig = px.bar(df, 
                x=data.keys(),
                y=data.values())
    return fig
    

In [323]:
def grouped_categorical_ct(df:pd.DataFrame, grouper:str, counter:str, colors:list=None):
    '''
    Chart two categorical variables, where one is a grouping condition 
    and the other is counted (frequency).
    Ex. Split data into those with and without heart disease. Then, 
    count how many men and women are in each group.

    Input: the dataframe, the name of the field to group by, and the 
    name of the field to count. Optional: list of bar colors in order
    Output: grouped bar chart
    '''
    df = df.groupby(grouper).agg(Count=(counter, 'value_counts')).reset_index()

    #ensures that bars appear in consistent order across charts
    df = df.sort_values([counter],ascending=[False]) 
    
    df[grouper] = df[grouper].replace(mappings[grouper])
    df[counter] = df[counter].replace(mappings[counter])

    fig = px.bar(df, 
                x=grouper,
                y="Count",
                color_discrete_sequence=colors,
                color=counter,
                barmode='group')
    return fig

In [324]:
df = pd.read_csv('data/heart_disease_prediction.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [325]:
print(f"Features: {len(df.columns)}")
print(f"Observations: {len(df)}")

Features: 12
Observations: 918


In [326]:
# Cast the following binary fields from int to string so they are treated as categorical values
df['FastingBS'] = df['FastingBS'].astype('str')
df['HeartDisease'] = df['HeartDisease'].astype('str')

In [327]:
df.describe(include='all')

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
count,918.000000,918,918,918.000000,918.000000,918,918,918.000000,918,918.000000,918,918
unique,NaN,2,4,NaN,NaN,2,3,NaN,2,NaN,3,2
top,NaN,M,ASY,NaN,NaN,0,Normal,NaN,N,NaN,Flat,1
freq,NaN,725,496,NaN,NaN,704,552,NaN,547,NaN,460,508
mean,53.510893,NaN,NaN,132.396514,198.799564,NaN,NaN,136.809368,NaN,0.887364,NaN,NaN
std,9.432617,NaN,NaN,18.514154,109.384145,NaN,NaN,25.460334,NaN,1.066570,NaN,NaN
min,28.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,60.000000,NaN,-2.600000,NaN,NaN
25%,47.000000,NaN,NaN,120.000000,173.250000,NaN,NaN,120.000000,NaN,0.000000,NaN,NaN
50%,54.000000,NaN,NaN,130.000000,223.000000,NaN,NaN,138.000000,NaN,0.600000,NaN,NaN
75%,60.000000,NaN,NaN,140.000000,267.000000,NaN,NaN,156.000000,NaN,1.500000,NaN,NaN


## Observations ##
Patient ages range from 28 to 77, with a mean of about 53 years.

It appears that RestingBP and Cholesterol have missing values, as their minimums are 0.0--a value that does not make sense for those measurements.

MaxHR is more ambiguous.  There is a minimum value of 60BPM, but the lower quartile is 120BPM. This may mean that 60 is a placeholder value for unrecorded heartrate. If these heartrates were achieved during exercise, it is unlikley that 60BPM was a recorded value.

The cholesterol column has a max value of 603, which is 2.5 times larger than what is considered "high" cholesterol.  Based on the IQR, this is an extreme outlier.  According to the American Heart Association, such a cholesterol level is caused by a rare genetic disease called familial hypercholesterolemia.


In [328]:
fig = categorical_ct(df, 'Sex')
fig.update_layout(
    title="Sex of Patients",
    xaxis_title="Sex",
    yaxis_title="Count of Patients")


There are more male patients represented in the data than female patients (~80% men)

In [329]:
fig = categorical_ct(df, 'ChestPainType')
fig.update_layout(
    title="Type of Patient Chest Pain",
    xaxis_title="Chest Pain Type",
    yaxis_title="Count of Patients")

Asymptomatic chest pain (meaning no chest pain) is the most common in the data.

In [330]:
fig = categorical_ct(df, 'HeartDisease')
fig.update_layout(
    title="Heart Disease Cases",
    xaxis_title="Test Result",
    yaxis_title="Count of Patients")

The dataset is pretty evenly split between patients with heart disease and those without heart disease. This means we have ample data for both classes, and we should be able to classify a test patient into one of the two categories. 

In [331]:
fig = categorical_ct(df, 'FastingBS')
fig.update_layout(
    title="Patient Blood Sugar Levels",
    xaxis_title="Blod Sugar Range",
    yaxis_title="Count of Patients")

The data is already split into categorical blood sugar ranges--above or below 120mg/dl.  A quick search revealed that the normal range is under 120mg/dl.  Most patients in the data are therefore non-diabetic.

In [332]:

fig = categorical_ct(df, 'RestingECG')
fig.update_layout(
    title='Patient Resting Electrocardiogram Results',
    xaxis_title="Test Result",
    yaxis_title="Count of Patients")

Most patients have a "normal" ECG test result. There are two categories that are therefore "abnormal".  If we combine the two abnormal groups, then we have a much closer split between normal and abnormal ECG readings.

In [333]:

fig = categorical_ct(df, 'ExerciseAngina')
fig.update_layout(
    title='Presense of Exercise-Induced Angina',
    xaxis_title="Angina Present?",
    yaxis_title="Count of Patients")


Most patients do not experience angina after exercise, but the split is very close.  It is unclear how this metric relates to the "ChestPainType" data, where 

In [334]:
fig = categorical_ct(df, 'ST_Slope')
fig.update_layout(
    title='Patients\' ECG ST-Segment Slope',
    xaxis_title="Test Result",
    yaxis_title="Count of Patients")

In [335]:
fig = grouped_categorical_ct(df, grouper='Sex', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by Sex',
    xaxis_title="Sex",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")


In [336]:
fig = grouped_categorical_ct(df, grouper='ChestPainType', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Presense of Heart Disease By Chest Pain Type',
    xaxis_title="Chest Pain Type",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

77% of all heart disease cases had no chest pain symptoms. Strangely, more healthy patients reported chest pain than patients with heart disease.  Non-anginal pain was the most common type of chest pain reported by patients with heart disease.  The most common pain reported by healthy patients was atypical angina. Typical angina was the least reported chest pain type overall, with almost equal numbers of healthy and heart disease patients.

In [337]:
fig = grouped_categorical_ct(df, grouper='FastingBS', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by Blood Sugar Level',
    xaxis_title="Blood Sugar Range",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

Non-diabetic patients are more common than diabetic patients overall.  Among non-diabetic patients, there are about the same number of patients with and without heart disease.  However, there is much larger portion of diabetic patients that also have heart disease.  While the absense of diabetes does not imply the absense of heart disease, the presense of diabetes is a strong indicator of heart disease.

In [338]:
fig = grouped_categorical_ct(df, grouper='RestingECG', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by ECG Result',
    xaxis_title="ECG Reading",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

There are fairly similar distributions of healthy and heart disease patients across the ECG reading categories.  The ST-T wave abnormality group has the most drastic difference in frequency.  It is more common for a person with heart disease to display an ST-T wave abnormality than a healthy patient, which could indicate another symptom or risk factor for heart disease.

In [339]:
fig = grouped_categorical_ct(df, grouper='ExerciseAngina', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Exercise-Induced Angina and Heart Disease',
    xaxis_title="Angina Induced by Exercise?",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

Most patients that have heart disease experience exercised-induced angina, and very few healthy patients display this symptom.  This may be a good feature to use in our predictive model. It is important to note that there is still a significant portion of patients with heart disease that do not experience angina after exercising (false negative risk).

In [340]:
fig = grouped_categorical_ct(df, grouper='ST_Slope', counter='HeartDisease', colors=['red', 'green'])
fig.update_layout(
    title='Heart Disease in Patients by ST-Segment Slope',
    xaxis_title="ST-Segment Slope",
    yaxis_title="Count of Patients",
    legend_title="Heart Disease Result")

After researching this measurement, I found that an upward slope of the ST-segment of an ECG reading is considered normal.  Downward or horizontal (flat) slopes are considered ST-segment depressions.

A large majority of those with heart disease display a flat ST-segment slope, while most healthy patients display a normal upward slope.  There are fewer patients overall that display a downward slope, but this is more common among those with heart disease rather than healthy individuals. Some healthy patients display a flat ST-segment slope. This could be an indicator that the patient is at risk of heart disease given that the upslope is more common among healthy individuals. More grouping analysis is needed for this metric, but it could make another strong feature for my model.

In [341]:
# Cleaning the values that don't make sense: RestingBP and Cholesterol
df[df['RestingBP']==0]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
449,55,M,NAP,0,0,0,Normal,155,N,1.5,Flat,1


Only one patient has a recorded resting BP of 0, so we will remove that record.

In [342]:
df = df[df['RestingBP'] != 0]

In [343]:
df[df['Cholesterol']==0]

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
293,65,M,ASY,115,0,0,Normal,93,Y,0.0,Flat,1
294,32,M,TA,95,0,1,Normal,127,N,0.7,Up,1
295,61,M,ASY,105,0,1,Normal,110,Y,1.5,Up,1
296,50,M,ASY,145,0,1,Normal,139,Y,0.7,Flat,1
297,57,M,ASY,110,0,1,ST,131,Y,1.4,Up,1
...,...,...,...,...,...,...,...,...,...,...,...,...
514,43,M,ASY,122,0,0,Normal,120,N,0.5,Up,1
515,63,M,NAP,130,0,1,ST,160,N,3.0,Flat,0
518,48,M,NAP,102,0,1,ST,110,Y,1.0,Down,1
535,56,M,ASY,130,0,0,LVH,122,Y,1.0,Flat,1


There are significantly more patients without a cholesterol reading.  We will fill these values with the median of the patient's sex, age group, and heart disease status.

In [344]:
age_bins = list(range(0, 100, 5))
df['AgeRange'] = pd.cut(df['Age'], bins=age_bins)
df_reduced = df[df['Cholesterol']!=0]
medians = df_reduced.groupby(['Sex', 'AgeRange', 'HeartDisease'], observed=True).agg(MedCholesterol=('Cholesterol', 'median'))
df = df.merge(medians, how='left', on=['Sex', 'AgeRange', 'HeartDisease'])
df['Cholesterol'] = np.where(df['Cholesterol'] == 0, df['MedCholesterol'], df['Cholesterol'])
df.drop(columns=['MedCholesterol'])


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,AgeRange
0,40,M,ATA,140,289.0,0,Normal,172,N,0.0,Up,0,"(35, 40]"
1,49,F,NAP,160,180.0,0,Normal,156,N,1.0,Flat,1,"(45, 50]"
2,37,M,ATA,130,283.0,0,ST,98,N,0.0,Up,0,"(35, 40]"
3,48,F,ASY,138,214.0,0,Normal,108,Y,1.5,Flat,1,"(45, 50]"
4,54,M,NAP,150,195.0,0,Normal,122,N,0.0,Up,0,"(50, 55]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,45,M,TA,110,264.0,0,Normal,132,N,1.2,Flat,1,"(40, 45]"
913,68,M,ASY,144,193.0,1,Normal,141,N,3.4,Flat,1,"(65, 70]"
914,57,M,ASY,130,131.0,0,Normal,115,Y,1.2,Flat,1,"(55, 60]"
915,57,F,ATA,130,236.0,0,LVH,174,N,0.0,Flat,1,"(55, 60]"
